In [3]:

import requests
import pandas as pd

In [4]:
#downloading page
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [5]:
#Please note, I am using the pandas function "read_html" we can easily process the HTML string instead of beautifulsoup
#We set header = 0 to use the first row as column names
#Since we will discard the "Not Assigned" columns, we set them to NaN so we can later use the dropna method.
df_html = pd.read_html(url)[0]



In [11]:
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for key, value in df_html.iteritems(): 
    #print('value', value)
    #print ('key', key)
    
    for i in range(len(value)):
        temp=value[i]
        if 'assigned' in temp:
            continue
        #print (temp)
        pc=temp[0:3]
        #print (pc)
        neiStartId=temp.find('(')
        neiEndId=temp.find(')')
        br=temp[3:neiStartId]
        #print (br)
        stNei=temp[neiStartId+1:neiEndId]
        #print (stNei)
        #lsNei=stNei.split('/')
        #print (lsNei)

        Postcode_u.append(pc)
        Borough_u.append(br)
        nei=stNei.replace(' ','')
        nei=nei.replace('/',',')
        Neighbourhood_u.append(nei)
            #print (pc, br, nei)
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.shape

(103, 3)

In [12]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek"
2,M1E,Scarborough,"Guildwood,Morningside,WestHill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df_toronto.loc[df_toronto['Postcode']=='M6A']

,Postcode,Borough,Neighbourhood
71,M6A,North York,"LawrenceManor,LawrenceHeights"


In [17]:

#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_toronto.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [20]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_toronto, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,WestHill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [21]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,HillcrestVillage,43.803762,-79.363452
57,M5G,Downtown Toronto,CentralBayStreet,43.657952,-79.387383


In [22]:
#Export to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [23]:
import pandas as pd
import numpy as np
import folium
import requests
from sklearn.cluster import KMeans

In [25]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_neighborhoods_coordinates['Borough'].unique()),
        df_neighborhoods_coordinates.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [26]:
df_neighborhoods_coordinates.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [27]:

df_neighborhoods_coordinates.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto                                                  9
Downtown Toronto                                                17
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East Toronto                                                     4
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East York                                                        4
East YorkEast Toronto                                            1
Etobicoke                                                       11
EtobicokeNorthwest                                               1
MississaugaCanada Post Gateway Processing Centre                 1
North York                                                      24
Queen's Park / Ontario Provincial Governmen                      1
Scarborough                                                     17
West Toronto                                                     6
York                                                  

In [28]:
df_toronto = df_neighborhoods_coordinates[df_neighborhoods_coordinates['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,TheBeaches,43.676357,-79.293031
1,M4J,East YorkEast Toronto,TheDanforthEast,43.685347,-79.338106
2,M4K,East Toronto,"TheDanforthWest,Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"IndiaBazaar,TheBeachesWest",43.668999,-79.315572
4,M4M,East Toronto,StudioDistrict,43.659526,-79.340923


In [29]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto                                                  9
Downtown Toronto                                                17
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East Toronto                                                     4
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
West Toronto                                                     6
Name: Neighborhood, dtype: int64


In [30]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [31]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.667725897435886, -79.38855562564102


In [32]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [33]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

Using Foursquare data

In [34]:
CLIENT_ID = 'E1E5SCKCSW04WBLCOHNZLJFZBCUDOZ3AV4O0HD1S0QUGJ2FT' # your Foursquare ID
CLIENT_SECRET = '5XVDSOHZWZM14HVJVKJO4FJ4KVHB21V5XQKTQNDOJMTFZENB' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

TheBeaches
TheDanforthEast
TheDanforthWest,Riverdale
IndiaBazaar,TheBeachesWest
StudioDistrict
LawrencePark
DavisvilleNorth
NorthTorontoWest
Davisville
MoorePark,SummerhillEast
SummerhillWest,Rathnelly,SouthHill,ForestHillSE,DeerPark
Rosedale
St.JamesTown,Cabbagetown
ChurchandWellesley
RegentPark,Harbourfront
GardenDistrict,Ryerson
St.JamesTown
BerczyPark
CentralBayStreet
Richmond,Adelaide,King
HarbourfrontEast,UnionStation,TorontoIslands
TorontoDominionCentre,DesignExchange
CommerceCourt,VictoriaHotel
Roselawn
ForestHillNorth&West
TheAnnex,NorthMidtown,Yorkville
UniversityofToronto,Harbord
KensingtonMarket,Chinatown,GrangePark
CNTower,KingandSpadina,RailwayLands,HarbourfrontWest,BathurstQuay,SouthNiagara,Islandairport
EnclaveofM5E
FirstCanadianPlace,Undergroundcity
Christie
Dufferin,DovercourtVillage
LittlePortugal,Trinity
Brockton,ParkdaleVillage,ExhibitionPlace
HighPark,TheJunctionSouth
Parkdale,Roncesvalles
Runnymede,Swansea
EnclaveofM4L


In [37]:
#Check size of resulting dataframe
toronto_venues.shape

(1675, 7)

In [38]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,TheBeaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,TheBeaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,TheBeaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,TheBeaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,TheDanforthEast,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park


In [39]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BerczyPark,55,55,55,55,55,55
"Brockton,ParkdaleVillage,ExhibitionPlace",22,22,22,22,22,22
"CNTower,KingandSpadina,RailwayLands,HarbourfrontWest,BathurstQuay,SouthNiagara,Islandairport",14,14,14,14,14,14
CentralBayStreet,81,81,81,81,81,81
Christie,17,17,17,17,17,17
ChurchandWellesley,88,88,88,88,88,88
"CommerceCourt,VictoriaHotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35
DavisvilleNorth,8,8,8,8,8,8


In [40]:

#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [41]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
toronto_onehot.shape

(1675, 238)

In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,BerczyPark,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton,ParkdaleVillage,ExhibitionPlace",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"CNTower,KingandSpadina,RailwayLands,Harbourfro...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.071429,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,CentralBayStreet,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,...,0.000000,0.00000,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.000000,0.00
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,ChurchandWellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,...,0.000000,0.00000,0.00,0.000000,0.000000,0.011364,0.000000,0.011364,0.011364,0.00
6,"CommerceCourt,VictoriaHotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,DavisvilleNorth,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,"Dufferin,DovercourtVillage",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.00


In [44]:
toronto_grouped.shape

(39, 238)

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BerczyPark,Coffee Shop,Café,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Eastern European Restaurant
1,"Brockton,ParkdaleVillage,ExhibitionPlace",Coffee Shop,Breakfast Spot,Café,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Performing Arts Venue
2,"CNTower,KingandSpadina,RailwayLands,Harbourfro...",Airport Service,Airport Lounge,Boat or Ferry,Coffee Shop,Harbor / Marina,Boutique,Airport Terminal,Bar,Airport Gate,Airport Food Court
3,CentralBayStreet,Coffee Shop,Italian Restaurant,Burger Joint,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Salad Place,Department Store,Gym / Fitness Center,Middle Eastern Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Candy Store,Diner,Italian Restaurant,Restaurant,Gas Station
5,ChurchandWellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Men's Store,Pub,Gastropub,Bubble Tea Shop,Dance Studio
6,"CommerceCourt,VictoriaHotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Gastropub,Japanese Restaurant,Seafood Restaurant,Deli / Bodega
7,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Gas Station,Seafood Restaurant
8,DavisvilleNorth,Park,Breakfast Spot,Gym,Gym / Fitness Center,Hotel,Food & Drink Shop,Department Store,Sandwich Place,Distribution Center,Diner
9,"Dufferin,DovercourtVillage",Bakery,Pharmacy,Pizza Place,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Middle Eastern Restaurant,Music Venue,Pool,Café


Clustering Neighborhoods

In [47]:
#using k-means square, we can classify neighborhoods

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [48]:
toronto_merged = df_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,TheBeaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
1,M4J,East YorkEast Toronto,TheDanforthEast,43.685347,-79.338106,0,Park,Convenience Store,Metro Station,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,M4K,East Toronto,"TheDanforthWest,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Grocery Store,Frozen Yogurt Shop,Pub,Pizza Place
3,M4L,East Toronto,"IndiaBazaar,TheBeachesWest",43.668999,-79.315572,0,Pub,Sushi Restaurant,Sandwich Place,Board Shop,Burrito Place,Italian Restaurant,Liquor Store,Restaurant,Fast Food Restaurant,Steakhouse
4,M4M,East Toronto,StudioDistrict,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop


In [49]:
#Visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# set color scheme for the clusters
cluster_color = {}
for cluster in range(kclusters):
    cluster_color[cluster]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    label_text = 'Cluster ' + str(cluster) + ' / ' + str(poi)
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=cluster_color[cluster],
        fill_color=cluster_color[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
#Examine cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
1,East YorkEast Toronto,0,Park,Convenience Store,Metro Station,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Grocery Store,Frozen Yogurt Shop,Pub,Pizza Place
3,East Toronto,0,Pub,Sushi Restaurant,Sandwich Place,Board Shop,Burrito Place,Italian Restaurant,Liquor Store,Restaurant,Fast Food Restaurant,Steakhouse
4,East Toronto,0,Café,Coffee Shop,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
5,Central Toronto,0,Park,Gym / Fitness Center,Swim School,Bus Line,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,Central Toronto,0,Park,Breakfast Spot,Gym,Gym / Fitness Center,Hotel,Food & Drink Shop,Department Store,Sandwich Place,Distribution Center,Diner
7,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Sporting Goods Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Chinese Restaurant
8,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Gas Station,Seafood Restaurant
9,Central Toronto,0,Park,Gym,Tennis Court,Playground,Women's Store,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center


In [51]:
# Count the number of neighborhoods per cluster
toronto_merged.groupby('Cluster Labels').count()['Neighborhood']

Cluster Labels
0    32
1     1
2     4
3     1
4     1
Name: Neighborhood, dtype: int64